In [2]:
from PIL import Image

In [3]:
image_path = "ryan.jpg"
img = Image.open(image_path)

In [4]:
width, height = img.size

#kuwahara filter time
for y in range(height):
    for x in range(width):
        r, g, b = img.getpixel((x, y))

        img.putpixel((x, y), (r, r, r))
img.save("ryan_modified.jpg")

In [5]:
from concurrent.futures import ThreadPoolExecutor

In [6]:
def get_mean_and_SD(img, x_min, x_max, y_min, y_max):
    mean = 0
    SD = 0
    for y in range(y_min, y_max):
        for x in range(x_min, x_max):
            mean += img.getpixel((x, y))[0]
    mean //= (x_max - x_min) * (y_max - y_min)
    for y in range(y_min, y_max):
        for x in range(x_min, x_max):
            SD += (img.getpixel((x, y))[0] - mean) ** 2
    SD /= (x_max - x_min) * (y_max - y_min)
    SD = SD ** 0.5
    return mean, SD

#now its a ryan_modified is a grayscale image
img = Image.open("ryan_modified.jpg")
#create a new image to store the filtered image
filtered_img = Image.new('RGB', (width, height), "black")
#odd sized square
square_size = 5
for y in range(square_size // 2, height - square_size // 2):
    for x in range(square_size // 2, width - square_size // 2):
        #iterate through each square
        mean_tl, sd_tl = get_mean_and_SD(img, x - square_size // 2, x, y - square_size // 2, y)
        mean_tr, sd_tr = get_mean_and_SD(img, x + 1, (x + 1) + square_size // 2, y - square_size // 2, y)
        mean_bl, sd_bl = get_mean_and_SD(img, x - square_size // 2, x, y + 1, (y + 1) + square_size // 2)
        mean_br, sd_br = get_mean_and_SD(img, x + 1, (x + 1) + square_size // 2, y + 1, (y + 1) + square_size // 2)
        #choose corresponding mean based on smallest SD
        mean = mean_tl
        if sd_tr <= sd_tl and sd_tr <= sd_bl and sd_tr <= sd_br:
            mean = mean_tr
        elif sd_bl <= sd_tl and sd_bl <= sd_tr and sd_bl <= sd_br:
            mean = mean_bl
        elif sd_br <= sd_tl and sd_br <= sd_tr and sd_br <= sd_bl:
            mean = mean_br
        #set pixel to mean
        filtered_img.putpixel((x, y), (mean, mean, mean))
filtered_img.save("ryan_kuwahara_filtered.jpg")